In [1]:
import os
import glob
import tensorflow as tf
from absl import logging
from modules import components, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

In [2]:
PIPELINE_NAME = "movie-recommender-pipeline"

# pipeline inputs
DATA_ROOT = "data/ratings"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [3]:
components = components.init_components(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    trainer_module=TRAINER_MODULE_FILE,
    train_steps=500,
    eval_steps=100,
    serving_model_dir=serving_model_dir,
    epochs=5,
)

pipeline = pipeline.init_pipeline(
    pipeline_root=pipeline_root,
    pipeline_name=PIPELINE_NAME,
    metadata_path=metadata_path,
    components=components,
)

BeamDagRunner().run(pipeline)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


In [ ]:
try:
    serving_model_dir = os.path.join(serving_model_dir, "movie-recommender")
    loaded = tf.saved_model.load(max(glob.glob(os.path.join(serving_model_dir, '*/')), key=os.path.getmtime))
    print(loaded({'userid_xf': [[42]], 'movieid_xf': [[15]]}).numpy()[0][0])
except BaseException as err:
    logging.error(f"ERROR IN model predict:\n{err}")